# Bulk functional analysis

Notebook imported from: https://decoupler-py.readthedocs.io/en/latest/notebooks/bulk.html. This is the complete tutorial for processing bulk RNA-seq, used in https://colab.research.google.com/drive/1llrWl4S5dpRlk7h6NxFnKiN-2I0UMAWn

Bulk RNA-seq yields many molecular readouts that are hard to interpret by themselves. One way of summarizing this information is by inferring pathway and transcription factor activities from prior knowledge.

In this notebook we showcase how to use `decoupler` for transcription factor (TF) and pathway activity inference from a human data-set. The data consists of 6 samples of hepatic stellate cells (HSC) where three of them were activated by the cytokine Transforming growth factor (TGF-β), it is available at GEO [here](https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSE151251).


> **Note:** This tutorial assumes that you already know the basics of `decoupler`. Else, check out the [Usage](https://decoupler-py.readthedocs.io/en/latest/notebooks/usage.html) tutorial first.



## Installing and loading packages

First, we need to load the relevant packages, `scanpy` to handle RNA-seq data
and `decoupler` to use statistical methods. We can install it using micromamba, in the available `saezlab` environment on the virtual machine with `micromamba install scanpy -c conda-forge`.

In [ ]:
!micromamba activate saezlab && micromamba install scanpy -c conda-forge

In [ ]:
import scanpy as sc
import decoupler as dc

# Only needed for processing
import numpy as np
import pandas as pd
from anndata import AnnData

## Loading the data

We can download the data easily from GEO:

In [ ]:
!wget 'https://www.ncbi.nlm.nih.gov/geo/download/?acc=GSE151251&format=file&file=GSE151251%5FHSCs%5FCtrl%2Evs%2EHSCs%5FTGFb%2Ecounts%2Etsv%2Egz' -O counts.txt.gz
!gzip -d -f counts.txt.gz

We can then read it using `pandas`:

In [ ]:
# Read raw data and process it
adata = pd.read_csv('counts.txt', index_col=2, sep='\t').iloc[:, 5:].T
adata

<div class="alert alert-info">

**Note**
    
In case your data does not contain gene symbols but rather gene ids like ENSMBL, you can use the function `sc.queries.biomart_annotations` to retrieve them.
In this other [vignette](https://decoupler-py.readthedocs.io/en/latest/notebooks/pseudobulk.html), ENSMBL ids are transformed into gene symbols.


</div>

<div class="alert alert-info">

**Note**
    
In case your data is not from human but rather a model organism, you can find their human orthologs using `pypath`.
Check this GitHub [issue](https://github.com/saezlab/decoupler-py/issues/5#issuecomment-1137099265) where mouse genes are transformed into human.


</div>

The obtained data consist of raw read counts for six different samples (three controls, three treatments) for ~60k genes.
Before continuing, we will transform our expression data into an `AnnData` object. It handles annotated data matrices
efficiently in memory and on disk and is used in the scverse framework. You can read more about it
[here](https://scverse.org/) and [here](https://anndata.readthedocs.io/en/latest/).

In [ ]:
# Transform to AnnData object
adata = AnnData(adata, dtype=np.float32)
adata.var_names_make_unique()
adata

Inside an `AnnData` object, there is the `.obs` attribute where we can store the metadata of our samples.
Here we will infer the metadata by processing the sample ids, but this could also be added from a separate dataframe:

In [ ]:
# Process treatment information
adata.obs['condition'] = ['control' if '-Ctrl' in sample_id else 'treatment' for sample_id in adata.obs.index]

# Process sample information
adata.obs['sample_id'] = [sample_id.split('_')[0] for sample_id in adata.obs.index]

# Visualize metadata
adata.obs

## Quality control

Before doing anything we need to ensure that our data passes some quality control thresholds. In transcriptomics it can happen
that some genes were not properly profiled and thus need to be removed.

To filter genes, we will follow the strategy implemented in the function `filterByExpr` from [edgeR](https://rdrr.io/bioc/edgeR/man/filterByExpr.html).
It keeps genes that have a minimum total number of reads across samples (`min_total_count`), and that have a minimum number of counts in a number of samples (`min_count`).

We can plot how many genes do we keep, you can play with the `min_count` and `min_total_count` to check how many genes would be kept when changed:

In [ ]:
dc.plot_filter_by_expr(adata, group=None, min_count=10, min_total_count=15, large_n=1, min_prop=1)

Here we can observe the frequency of genes with different total sum of counts and number of samples.
For this example, we have decided to keep genes that show some degree of expression in all our samples.

<div class="alert alert-info">

**Note**

This thresholds can vary a lot between datasets, manual assessment of them needs to be considered. For example, it might be
the case that many genes are not expressed in just one sample which they would get removed by the current setting. For this
specific dataset it is fine.

</div>

Once we are content with the threshold parameters, we can perform the actual filtering:

In [ ]:
# Obtain genes that pass the thresholds
genes = dc.filter_by_expr(adata, group=None, min_count=10, min_total_count=15, large_n=1, min_prop=1)

# Filter by these genes
adata = adata[:, genes].copy()
adata

## Differential expression analysis

In order to identify which are the genes that are changing the most between treatment and control we can perform differential
expression analysis (DEA). For this example, we will perform a simple experimental design where we compare the gene expression
of treated cells against controls. We will use the python implementation of the framework DESeq2, but we could have used any
other one (`limma` or `edgeR` for example). For a better understanding how it works, check
[DESeq2's documentation](https://pydeseq2.readthedocs.io/en/latest/). Note that more complex experimental designs can be used
by adding more factors to the `design_factors` argument.

In [ ]:
# Import DESeq2
from pydeseq2.dds import DeseqDataSet
from pydeseq2.ds import DeseqStats

In [ ]:
# Build DESeq2 object
dds = DeseqDataSet(
    adata=adata,
    design_factors='condition',
    refit_cooks=True,
    n_cpus=8,
)

In [ ]:
# Compute LFCs
dds.deseq2()

In [ ]:
# Extract contrast between COVID-19 vs normal
stat_res = DeseqStats(dds, contrast=["condition", 'treatment', 'control'], n_cpus=8)

In [ ]:
# Compute Wald test
stat_res.summary()

In [ ]:
stat_res.LFC

In [ ]:
# Shrink LFCs
stat_res.lfc_shrink('condition_treatment_vs_control')

In [ ]:
# Extract results
results_df = stat_res.results_df
results_df

In [ ]:
results_df.stat.hist(bins=100);

In [ ]:
results_df.to_csv('deseq2_covid19_vs_normal.csv')

We can plot the obtained results in a volcano plot:

In [ ]:
dc.plot_volcano_df(results_df, x='log2FoldChange', y='padj', top=20)

After performing DEA, we can use the obtained gene level statistics to perform enrichment analysis. Any statistic can be used,
but we recommend using the t-values instead of logFCs since t-values incorporate the significance of change in their value.
We will transform the obtained t-values stored in `stats` to a wide matrix so that it can be used by `decoupler`:

In [ ]:
mat = results_df[['stat']].T.rename(index={'stat': 'treatment.vs.control'})
mat

In [ ]:
mat.to_csv('deseq2_stat_covid19_vs_normal.csv')

## Pathway activity inference
To estimate pathway activities we will use the resource PROGENy and the `consensus` method.

For another example on pathway activities please visit this other notebook: [Pathway activity inference](https://decoupler-py.readthedocs.io/en/latest/notebooks/progeny.html).

In [ ]:
# Retrieve PROGENy model weights
progeny = dc.get_progeny(top=300)

# Infer pathway activities with consensus
pathway_acts, pathway_pvals = dc.run_consensus(mat=mat, net=progeny)
pathway_acts

Let us plot the obtained scores:

In [ ]:
dc.plot_barplot(pathway_acts, 'treatment.vs.control', top=25, vertical=False)

As expected, after treating cells with the cytokine TGFb we see an increase of activity for this pathway.

On the other hand, it seems that this treatment has decreased the activity of other pathways like JAK-STAT or TNFa.

We can visualize the targets of TFGb in a scatter plot:

In [ ]:
dc.plot_targets(results_df, stat='stat', source_name='TGFb', net=progeny, top=15)

The observed activation of TGFb is due to the fact that majority of its target genes with positive weights have positive
t-values (1st quadrant), and the majority of the ones with negative weights have negative t-values (3d quadrant).

## Transcription factor activity inference
Similarly to pathways, we can estimate transcription factor activities using the resource DoRothEA and the `consensus` method.

For another example on transcription factor activities please visit this other notebook: [Transcription factor activity inference](https://decoupler-py.readthedocs.io/en/latest/notebooks/dorothea.html).

In [ ]:
# Retrieve DoRothEA gene regulatory network
dorothea = dc.get_dorothea()

# Infer pathway activities with consensus
tf_acts, tf_pvals = dc.run_consensus(mat=mat, net=dorothea)
tf_acts

Let us plot the obtained scores for the top active/inactive transcription factors:

In [ ]:
dc.plot_barplot(tf_acts, 'treatment.vs.control', top=25, vertical=True)

SRF, E2F4 and EPAS1 seem to be the most activated in this treatment while STAT2, IRF1 AND MAFG seem to be inactivated.

As before, we can manually inspect the downstream targets of each transcription factor:

In [ ]:
dc.plot_targets(results_df, stat='stat', source_name='SRF', net=dorothea, top=15)

SRF seems to be active in treated cells since their positive targets are up-regulated.

If needed, we can also look at this at the volcano plot:

In [ ]:
# Extract logFCs and pvals
logFCs = results_df[['log2FoldChange']].T.rename(index={'log2FoldChange': 'treatment.vs.control'})
pvals = results_df[['padj']].T.rename(index={'padj': 'treatment.vs.control'})

# Plot
dc.plot_volcano(logFCs, pvals, 'treatment.vs.control', name='SRF', net=dorothea, top=10, sign_thr=0.05, lFCs_thr=0.5)